In [6]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
import keras as k

import sklearn

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

from keras.initializers import GlorotUniform

from matplotlib import pyplot as plt
import seaborn as sns
import scipy

from datetime import datetime

import pickle

from PIL import Image

print(f"pd.__version__: {pd.__version__}")
print(f"tf.__version__: {tf.__version__}")

image_size = (96, 128)

pd.__version__: 2.2.3
tf.__version__: 2.15.1


In [7]:
image_shape = (image_size[0], image_size[1], 3)

base_model = keras.applications.MobileNetV3Small(
    include_top=False,
    weights="imagenet",
    input_shape=image_shape,
    pooling=None,
    include_preprocessing=False, 
)
base_model.trainable = False

In [8]:
feature_extractor = base_model.output

print(feature_extractor.shape)

a_model = []

output_activation_1 = 'sigmoid'


# model 0
i = 0

a = feature_extractor
a = layers.Conv2D(384, (3, 3), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_2')(feature_extractor)
a = layers.BatchNormalization(name=f'a{i}_bn_2')(a)
a = layers.Activation('relu', name=f'a{i}_relu_2')(a)
a = layers.Conv2D(128, (3, 3), padding='same', 
                  kernel_initializer='he_uniform', name=f'a{i}_conv_1')(feature_extractor)
a = layers.BatchNormalization(name=f'a{i}_bn_1')(a)
a = layers.Activation('relu', name=f'a{i}_relu_1')(a)
a = layers.GlobalAveragePooling2D(name=f'a{i}_GAP')(a)
a = layers.Dense(96, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_2')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout_2')(a)
a = layers.Dense(64, activation='relu', kernel_initializer='he_uniform', name=f'a{i}_dense_1')(a)
a = layers.Dropout(0.5, name=f'a{i}_dropout')(a)

# Branch 1: No arrow (regression, 0 to 1)
arrow_output = layers.Dense(3, activation="softmax", name='no_arrow_output')(a)

(None, 3, 4, 576)


In [9]:
model = Model(inputs=base_model.input, outputs=arrow_output, name="angle_model")

learning_rate = 0.0010

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss='mse',
    metrics=['accuracy'],
)

In [10]:
model.save_weights(f"{image_size[0]}x{image_size[1]}.weights.h5")